# 第6章モジュール

ここまで使ってきた関数、型、型クラスはデフォルトでimportされるPreludeというモジュールの一部

In [1]:
import Data.List

numUniques :: (Eq a) => [a] -> Int
numUniques = length . nub

In [2]:
:t nub

nub :: forall a. Eq a => [a] -> [a]

In [3]:
nub [1,2,3,4,1]

[1,2,3,4]

Hoogle ｗｗｗｗｗ  
https://www.haskell.org/hoogle/

In [4]:
:m + Data.List Data.Map Data.Set

In [7]:
import Data.List (nub, sort) -- 一部をimport

In [8]:
nub [1,2,3,4,5,1]

[1,2,3,4,5]

In [11]:
import Data.List hiding (nub) -- importから除く場合

In [10]:
nub [1,2,3,4,5,1]

Not in scope: `nub'

In [12]:
import qualified Data.Map -- こうするとData.Map.filterと使う必要がある
import qualified Data.Map as M -- こうすれば M.filterと使うことが出来る

### M.filterと関数合成演算子の違いは

`.`がqualified されたモジュール名と関数名の間に空白を開けずに置いた場合、importされた関数であるとみなされ、そうじゃなければ関数合成としてみなされる

## 標準モジュールの環数で問題を解く

In [13]:
import Data.List (words)

words "hey these are the words in this sentence"

["hey","these","are","the","words","in","this","sentence"]

In [14]:
words "hey these                   are the words in the sentence"

["hey","these","are","the","words","in","the","sentence"]

In [17]:
import Data.List (group)

group [1,1,1,1,12,2,2,2,2,23,4,4,3,3,4,34,3,5,3,1,5,6,5,89,1,1,34,4,34]

[[1,1,1,1],[12],[2,2,2,2],[23],[4,4],[3,3],[4],[34],[3],[5],[3],[1],[5],[6],[5],[89],[1,1],[34],[4],[34]]

In [18]:
import Data.List (sort)

sort [ 5,4,76,3,1,4,5,6]

[1,3,4,4,5,5,6,76]

In [25]:
import Data.Set hiding (map)
import Data.Map hiding (map)
import Data.List (map)
wordNums :: String -> [(String, Int)]
wordNums = map (\ws -> (head ws, length ws)) . D.group . D.sort . D.words

Line 2: Use &&&
Found:
\ ws -> (head ws, length ws)
Why not:
head Control.Arrow.&&& length

In [26]:
wordNums  "hey these are the words in this sentence"

[("are",1),("hey",1),("in",1),("sentence",1),("the",1),("these",1),("this",1),("words",1)]

In [27]:
import Data.List

isIn :: (Eq a) => [a] -> [a] -> Bool
needle `isIn` haystack = any (needle `isPrefixOf` ) (tails haystack)

In [28]:
"art" `isIn` "party"

True

In [29]:
[1,2] `isIn` [1,3,5]

False

In [30]:
"art" `isInfixOf` "party"

True

### import参考

https://wiki.haskell.org/Import

## シーザー暗号サラダ

次回乞うご期待